# Notebook for QC addressing data quality, column format and business rules

### Note
This notebook deals with demo of QC module - address issues and additions to <a>ashoksubbu99@gmail.com</a>.
### Sample scenario - on how to use this module

**Scenario: ** List the type of customers whose spend is less than lower_bound <br/>
**Steps: **
    * data_quality(data_frame,["spend_value","type"],"summary") - get summary of customer types and spend_value
    * agg_check(data_frame,"spend_value",[lower_bound,upper_bound],'range') - creates file for spend less than lower bound
    * check_csv = pd.read_csv("path_to_generated_file") - reads the above generated file
    * agg_check(check_csv,"type",[],'levels') - returns the type of columns

### Demo with generic transaction data - covers important aspects of the module
#### Importing required modules and dataset to perform QC

In [2]:
from QC_rules import validate_book_on_load, data_quality, groupby_check, format_check, agg_check, Check_with_time, notebook_summary
import pandas as pd

# Please make note of the format of path to the file
file_path = "/home/musigma/Desktop/python_qc_brick/data/transaction.csv"
final_time_frame = pd.DataFrame(columns=['Test','Result','Time'])

data_frame = validate_book_on_load(file_path)
if isinstance(data_frame, pd.DataFrame):
    print(data_frame.columns)
else:
    print(data_frame)

Index([u'customer_id', u'item_id', u'bill_number', u'quantity', u'spend_value',
       u'purchase_date', u'file_name'],
      dtype='object')


### Data Quality module -  returns NULL %, unique %, summary based on type of columns
* **For NULL % of columns -** ```data_quality(data_frame,['col1','col2','col3'....],"NULL")``` <br/>
* **For unique % of columns -** ``` data_quality(data_frame,['col1','col2','col3'....],"unique")``` <br/>
* **For summary of columns -** ``` data_quality(data_frame,['col1','col2','col3'....],"summary")``` <br/>

In [3]:
#data_quality(data_frame,list(data_frame.columns),"summary")
time_frame, quality_output = Check_with_time(data_frame,"quality",list(data_frame.columns),[],[],"unique")
final_time_frame =  final_time_frame.append(time_frame)
quality_output

{'bill_number': '99.5%',
 'customer_id': '14.5%',
 'file_name': '6.0%',
 'item_id': '82.5%',
 'purchase_date': '100.0%',
 'quantity': '2.0%',
 'spend_value': '47.0%'}

### Format Check module - checks if the column is following particular format
** Please find sample formats below, Complete regex reference <a href="https://www.debuggex.com/cheatsheet/regex/python">here</a> ** <br/>

* Word - '\w+'
* Number - '\d+'
* Email - '[^@]+@[^@]+\.[^@]+'
* Double space - ' +'
* Year - '%Y-%m-%d' if year in (yyyy-mm-dd) <br/>

If in doubt, use *data_frame[column].head(10)* to check for sample rows of column

In [4]:
#format_check(data_frame,"processing_date","%m/%d/%Y")
time_frame, format_output = Check_with_time(data_frame,"format","purchase_date",[],[],"%m/%d/%Y")
final_time_frame =  final_time_frame.append(time_frame)
format_output

True

### Aggregate module - Range checks at a column level 
* **Get count at different levels of column -** ```agg_check(data_frame,"column",[],"levels")```
* **If column is boun by range of values -** ```agg_check(data_frame,"column",[lower_bound,upper_bound],"range")```
    * values to be in numeric,date (yyyy-mm-dd) format
    * If test fails, saves the csv file with error rows to current working directory
* ** Check if aggregate of a column is bound by range - **```agg_check(data_frame,"column",[lower_bound,upper_bound],argument)```
    * currently, argument supports "sum" and "count" checks
    * supports only numeric columns


In [5]:
#agg_check(data_frame,"file_name",[],"levels")
time_frame, agg_output = Check_with_time(data_frame,"aggregate","file_name",[],[],"levels")
final_time_frame =  final_time_frame.append(time_frame)
agg_output

,file_name,count of file_name
0,April,16
1,August,16
2,December,17
3,February,17
4,January,17
5,July,16
6,June,16
7,March,17
8,May,16
9,November,17


In [6]:
#agg_check(data_frame,"spend_value",[200,400],'range')
time_frame, agg_output = Check_with_time(data_frame,"aggregate","spend_value",[],[200,400],"range")
final_time_frame =  final_time_frame.append(time_frame)
agg_output

,Result,Test
0,False,spend_value range greater than 200
1,False,spend_value range less than 400


In [7]:
#agg_check(data_frame,"processing_date",['2018-08-02','2018-12-01'],'range')
time_frame, agg_output = Check_with_time(data_frame,"aggregate","purchase_date",[],['2018-08-02','2018-12-01'],"range")
final_time_frame =  final_time_frame.append(time_frame)
agg_output

,Result,Test
0,False,purchase_date range greater than 2018-08-02
1,False,purchase_date range less than 2018-12-01


In [8]:
#agg_check(data_frame,"spend_value",[100,20],"sum")
time_frame, agg_output = Check_with_time(data_frame,"aggregate","spend_value",[],[100,20],"sum")
final_time_frame =  final_time_frame.append(time_frame)
agg_output

,Result,Test
0,False,spend_value sum less than 100
1,True,spend_value sum greater than 20


### Groupby module - for aggregate stats of a column on group by of selected columns
* ** with group of selected columns -** ``` groupby_check(data_frame,[col1,col2....],[col1,col2,col3.....],function)```
* ** At column level (with no groupby) - ** ```groupby_check(data_frame,[col1,col2,col3...],[],function)```
    * supported functions are sum,count,mean
    * groupby_check syntax order : *groupby_check(data_frame, desired_column_list, groupby_columns_list, function)*

In [9]:
#groupby_check(data_frame,["spend_value","quantity"],["store_id"],"sum")
time_frame, group_output = Check_with_time(data_frame,"groupby",["spend_value","quantity"],["customer_id"],[],"sum")
final_time_frame =  final_time_frame.append(time_frame)
group_output

,spend_value,quantity
customer_id,,
75013356,241.5000,11
75013360,124.0000,7
75013362,50.0000,1
75013364,9.0000,1
75013370,10.0000,1
75013374,567.0000,17
75013376,167.0000,13
75013384,136.5000,9
75013666,12.0000,1


In [10]:
#groupby_check(data_frame,["spend_value","quantity"],[],"sum")
time_frame, group_output = Check_with_time(data_frame,"groupby",["spend_value","quantity"],[],[],"sum")
final_time_frame =  final_time_frame.append(time_frame)
group_output

,column,sum
0,spend_value,8011.8209
0,quantity,218.0000


In [11]:
final_time_frame.reset_index(drop=True)

,Result,Test,Time
0,Completed,"unique of ['customer_id', 'item_id', 'bill_num...",0.002545
1,True,check format of purchase_date is %m/%d/%Y,0.071668
2,Completed,levels of file_name,0.003190
3,False,spend_value range greater than 200,0.011870
4,False,spend_value range less than 400,0.011870
5,False,purchase_date range greater than 2018-08-02,0.318459
6,False,purchase_date range less than 2018-12-01,0.318459
7,False,spend_value sum less than 100,0.001731
8,True,spend_value sum greater than 20,0.001731
9,Completed,"sum of ['spend_value', 'quantity'] by ['custom...",0.004866


In [12]:
notebook_summary(file_path,final_time_frame)

,Name of file,Size of file(bytes),Total time
0,transaction.csv,10655,0.755978
